In [30]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
# To get model performance 
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
# loading everything that might be helpful
import sys
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (9, 6)
from functools import partial
import PIL
import PIL.Image
import torch
from torch.utils.data import DataLoader, random_split
from torch import Generator
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
from torch import manual_seed as torch_manual_seed
from torch.cuda import max_memory_allocated, set_device, manual_seed_all
from torch.backends import cudnn
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

from deeptables.models import deeptable, deepnets
from tensorflow.keras.optimizers import RMSprop

# Set seed for whole notebook
import random
random.seed(10)

In [31]:
#Loading locational dataset
df = pd.read_csv('../final-dataset/final-scaled-df.csv', index_col='FIPS')
df.head(20)

TEST_RATIO = 0.2
BATCH_SIZE = 64

size_all = len(df)

# training/val/test dataset
size_train = size_all - 6
size_val = int(size_train * TEST_RATIO)
size_train_sub = size_train - size_val

# split data into train and test
# Test = 6 counties of interst: Accomack, Fauquier, Greensville, Hanover, Rockingham, Wise
dataset_val = df.loc[[51001, 51061, 51081, 51085, 51165, 51195]]

# Train = all except 6 counties of interest
dataset_train = df.drop([51001, 51061, 51081, 51085, 51165, 51195])

cols = ['Income', 'DSCI', 'PQ1', 'PQ2', 'PQ3', 'PQ4', 'TQ1', 'TQ2', 'TQ3', 'TQ4', 'Forest', 
        'Agriculture', 'Biomass', 'Power', 'Water Availability', 'Watershed',
       'Log_Bio','Log_Water','Log_Power']
cols_rf = ['PQ1', 'PQ2', 'PQ3', 'PQ4', 'TQ1', 'TQ2', 'TQ3', 'TQ4', 'Forest','Water Availability', 'Log_Water']
cols_ew = ['PQ1', 'PQ2', 'PQ3', 'PQ4', 'TQ1', 'TQ2', 'TQ3', 'TQ4', 'Income', 'Power', 'Log_Power', 'Agriculture']
cols_bio = ['PQ1', 'PQ2', 'PQ3', 'PQ4', 'TQ1', 'TQ2', 'TQ3', 'TQ4', 'Income', 'Biomass', 'Log_Bio','Agriculture']

In [32]:
# Using the elbow method to determine the k value to be applied
k_rng = range(1, 10)
sse = []
for k in k_rng:
    km = KMeans(n_clusters=k, n_init=10)
    km.fit(df[['PQ1', 'PQ2', 'PQ3', 'PQ4', 'TQ1', 'TQ2', 'TQ3', 'TQ4', 'Forest','Log_Water']])
    sse.append(km.inertia_)

# create clusters using k value = 4
hc = AgglomerativeClustering(n_clusters=4, affinity = 'euclidean', linkage = 'ward')

# Determining mean cluster characterisitics REFORESTATION
y_hc = hc.fit_predict(dataset_train[cols_rf])
dataset_train['Reforest'] = y_hc
dataset_train.sort_values("Reforest", inplace = True, ascending=True)

#average input vars by cluster
df_rfcluster = dataset_train.groupby('Reforest').mean()

# Determining mean cluster characterisitics ENHANCED WEATHERING
y_hc = hc.fit_predict(dataset_train[cols_ew])
dataset_train['EW'] = y_hc
dataset_train.sort_values("EW", inplace = True, ascending=True)

#average input vars by cluster
df_ewcluster = dataset_train.groupby('EW').mean()

# Determining mean cluster characterisitics BIOCHAR
y_hc = hc.fit_predict(dataset_train[cols_bio])
dataset_train['Biochar'] = y_hc
dataset_train.sort_values("Biochar", inplace = True, ascending=True)

#average input vars by cluster
df_biocluster = dataset_train.groupby('Biochar').mean()

In [33]:
# creating input identity
sub_df = dataset_train[cols]
X = sub_df
y = dataset_train[['Reforest','EW','Biochar']]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1)

X_train_rf = X_train[cols_rf]
X_train_ew = X_train[cols_ew]
X_train_bio = X_train[cols_bio]

X_test_rf = X_test[cols_rf]
X_test_ew = X_test[cols_ew]
X_test_bio = X_test[cols_bio]

y_train_rf = y_train[['Reforest']]
y_train_ew = y_train[['EW']]
y_train_bio = y_train[['Biochar']]

y_test_rf = y_test[['Reforest']]
y_test_ew = y_test[['EW']]
y_test_bio = y_test[['Biochar']]

dataset_val_rf = dataset_val[cols_rf]
dataset_val_ew = dataset_val[cols_ew]
dataset_val_bio = dataset_val[cols_bio]

In [34]:
# Make data into tensor objects
X_train_rf_tensor = torch.tensor(X_train_rf.values, dtype=torch.float32)
X_train_ew_tensor = torch.tensor(X_train_ew.values, dtype=torch.float32)
X_train_bio_tensor = torch.tensor(X_train_bio.values, dtype=torch.float32)

X_test_rf_tensor = torch.tensor(X_test_rf.values, dtype=torch.float32)
X_test_ew_tensor = torch.tensor(X_test_ew.values, dtype=torch.float32)
X_test_bio_tensor = torch.tensor(X_test_bio.values, dtype=torch.float32)

y_train_rf_tensor = torch.tensor(y_train_rf['Reforest'].values, dtype=torch.float32)
y_train_ew_tensor = torch.tensor(y_train_ew['EW'].values, dtype=torch.float32)
y_train_bio_tensor = torch.tensor(y_train_bio['Biochar'].values, dtype=torch.float32)

y_test_rf_tensor = torch.tensor(y_test_rf['Reforest'].values, dtype=torch.float32)
y_test_ew_tensor = torch.tensor(y_test_ew['EW'].values, dtype=torch.float32)
y_test_bio_tensor = torch.tensor(y_test_bio['Biochar'].values, dtype=torch.float32)

# Make test data into tensor objects
X_val_rf_tensor = torch.tensor(dataset_val_rf.values, dtype=torch.float32)
X_val_ew_tensor = torch.tensor(dataset_val_ew.values, dtype=torch.float32)
X_val_bio_tensor = torch.tensor(dataset_val_bio.values, dtype=torch.float32)

#train data
training_data_rf = TensorDataset(X_train_rf_tensor, y_train_rf_tensor)
train_dataloader_rf = DataLoader(training_data_rf, batch_size=64)

training_data_ew = TensorDataset(X_train_ew_tensor, y_train_ew_tensor)
train_dataloader_ew = DataLoader(training_data_ew, batch_size=64)

training_data_bio = TensorDataset(X_train_bio_tensor, y_train_bio_tensor)
train_dataloader_bio = DataLoader(training_data_bio, batch_size=64)


#test data
test_data_rf = TensorDataset(X_test_rf_tensor,  y_test_rf_tensor)
test_dataloader_rf = DataLoader(test_data_rf, batch_size=64)

test_data_ew = TensorDataset(X_test_ew_tensor,  y_test_ew_tensor)
test_dataloader_ew = DataLoader(test_data_ew, batch_size=64)

test_data_bio = TensorDataset(X_test_bio_tensor,  y_test_bio_tensor)
test_dataloader_bio = DataLoader(test_data_bio, batch_size=64)


#validate data
val_data_rf = TensorDataset(X_val_rf_tensor)
val_dataloader_rf = DataLoader(val_data_rf, batch_size=64)

val_data_ew = TensorDataset(X_val_ew_tensor)
val_dataloader_ew = DataLoader(val_data_ew, batch_size=64)

val_data_bio = TensorDataset(X_val_bio_tensor)
val_dataloader_bio = DataLoader(val_data_bio, batch_size=64)

In [35]:
# conf = deeptable.ModelConfig(nets=['dnn_nets'], optimizer=RMSprop())

conf = deeptable.ModelConfig(
    metrics=['RootMeanSquaredError'], 
    nets=deepnets.DCN,
    dnn_params={
        'hidden_units': ((256, 0.3, True), (256, 0.3, True)),
        'dnn_activation': 'relu',
    },
    earlystopping_patience=5,
)

In [36]:
dt_rf = deeptable.DeepTable(config=conf)
dt_rf.fit(X_train_rf_tensor, y_train_rf['Reforest'], epochs=100)
score = dt_rf.evaluate(X_test_rf_tensor, y_test_rf['Reforest'], batch_size=512, verbose=0)
preds = dt_rf.predict(X_test_rf_tensor)
score

04-11 16:46:58 I deeptables.m.deeptable.py 337 - X.Shape=torch.Size([67, 11]), y.Shape=(67,), batch_size=128, config=ModelConfig(name='conf-1', nets=['dcn_nets'], categorical_columns='auto', exclude_columns=[], task='auto', pos_label=None, metrics=['RootMeanSquaredError'], auto_categorize=False, cat_exponent=0.5, cat_remain_numeric=True, auto_encode_label=True, auto_imputation=True, auto_discrete=False, auto_discard_unique=True, apply_gbm_features=False, gbm_params={}, gbm_feature_type='embedding', fixed_embedding_dim=True, embeddings_output_dim=4, embeddings_initializer='uniform', embeddings_regularizer=None, embeddings_activity_regularizer=None, dense_dropout=0, embedding_dropout=0.3, stacking_op='add', output_use_bias=True, apply_class_weight=False, optimizer='auto', loss='auto', dnn_params={'hidden_units': ((256, 0.3, True), (256, 0.3, True)), 'dnn_activation': 'relu'}, autoint_params={'num_attention': 3, 'num_heads': 1, 'dropout_rate': 0, 'use_residual': True}, fgcnn_params={'fg_f

04-11 16:46:58 W deeptables.m.preprocessor.py 154 - Column index of X has been converted: Index(['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9',
       'x_10'],
      dtype='object')


04-11 16:46:58 I deeptables.m.preprocessor.py 261 - Preparing features...
04-11 16:46:58 I deeptables.m.preprocessor.py 336 - Preparing features taken 0.013317584991455078s
04-11 16:46:58 I deeptables.m.preprocessor.py 341 - Data imputation...
04-11 16:46:59 I deeptables.m.preprocessor.py 383 - Imputation taken 0.012956380844116211s
04-11 16:46:59 I deeptables.m.preprocessor.py 388 - Categorical encoding...
04-11 16:46:59 I deeptables.m.preprocessor.py 393 - Categorical encoding taken 0.10834240913391113s
04-11 16:46:59 I deeptables.m.preprocessor.py 196 - fit_transform taken 0.15644478797912598s
04-11 16:46:59 I deeptables.m.deeptable.py 353 - Training...
04-11 16:46:59 I deeptables.m.deeptable.py 752 - Injected a callback [EarlyStopping]. monitor:val_rootmeansquarederror, patience:5, mode:min
04-11 16:46:59 I deeptables.u.dataset_generator.py 250 - create dataset generator with _TFDGForPandas, batch_size=128, shuffle=True, drop_remainder=True
04-11 16:46:59 I deeptables.u.dataset_gen

/home/hrn4ch/.conda/envs/myenv/lib/python3.7/site-packages/keras/initializers/initializers_v2.py:121: UserWarning: The initializer RandomUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "


04-11 16:46:59 I deeptables.m.deepmodel.py 287 - >>>>>>>>>>>>>>>>>>>>>> Model Desc <<<<<<<<<<<<<<<<<<<<<<< 
---------------------------------------------------------
inputs:
---------------------------------------------------------
['all_categorical_vars: (11)']
---------------------------------------------------------
embeddings:
---------------------------------------------------------
input_dims: [69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69]
output_dims: [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
dropout: 0.3
---------------------------------------------------------
dense: dropout: 0
batch_normalization: False
---------------------------------------------------------
concat_embed_dense: shape: (None, 44)
---------------------------------------------------------
nets: ['dcn_nets']
---------------------------------------------------------
dcn-widecross: input_shape (None, 44), output_shape (None, 44)
dcn-dnn2: input_shape (None, 44), output_shape (None, 256)
dcn: input_shape (None, 44), output_

04-11 16:47:05 W deeptables.m.preprocessor.py 154 - Column index of X has been converted: Index(['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9',
       'x_10'],
      dtype='object')


04-11 16:47:05 I deeptables.m.preprocessor.py 249 - transform_X taken 0.04326200485229492s
04-11 16:47:05 I deeptables.m.preprocessor.py 230 - Transform [y]...
04-11 16:47:05 I deeptables.m.preprocessor.py 236 - transform_y taken 0.0003292560577392578s
04-11 16:47:05 I deeptables.m.deepmodel.py 158 - Performing evaluation...
04-11 16:47:05 I deeptables.u.dataset_generator.py 250 - create dataset generator with _TFDGForPandas, batch_size=512, shuffle=False, drop_remainder=False
04-11 16:47:05 I deeptables.m.deeptable.py 685 - Perform prediction...
04-11 16:47:05 I deeptables.m.preprocessor.py 242 - Transform [X]...


04-11 16:47:05 W deeptables.m.preprocessor.py 154 - Column index of X has been converted: Index(['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9',
       'x_10'],
      dtype='object')


04-11 16:47:05 I deeptables.m.preprocessor.py 249 - transform_X taken 0.039322853088378906s
04-11 16:47:05 I deeptables.m.deepmodel.py 130 - Performing predictions...
04-11 16:47:05 I deeptables.u.dataset_generator.py 250 - create dataset generator with _TFDGForPandas, batch_size=128, shuffle=False, drop_remainder=False
04-11 16:47:06 I deeptables.m.deeptable.py 559 - predict_proba taken 0.2834320068359375s
04-11 16:47:06 I deeptables.m.deeptable.py 594 - Reverse indicators to labels.


{'loss': 1.2133923768997192, 'root_mean_squared_error': 0.41710054874420166}

In [37]:
dt_ew = deeptable.DeepTable(config=conf)
dt_ew.fit(X_train_ew_tensor, y_train_ew['EW'], epochs=100)
score = dt_ew.evaluate(X_test_ew_tensor, y_test_ew['EW'], batch_size=512, verbose=0)
preds = dt_ew.predict(X_test_ew_tensor)
score

04-11 16:47:22 I deeptables.m.deeptable.py 337 - X.Shape=torch.Size([67, 12]), y.Shape=(67,), batch_size=128, config=ModelConfig(name='conf-1', nets=['dcn_nets'], categorical_columns='auto', exclude_columns=[], task='auto', pos_label=None, metrics=['RootMeanSquaredError'], auto_categorize=False, cat_exponent=0.5, cat_remain_numeric=True, auto_encode_label=True, auto_imputation=True, auto_discrete=False, auto_discard_unique=True, apply_gbm_features=False, gbm_params={}, gbm_feature_type='embedding', fixed_embedding_dim=True, embeddings_output_dim=4, embeddings_initializer='uniform', embeddings_regularizer=None, embeddings_activity_regularizer=None, dense_dropout=0, embedding_dropout=0.3, stacking_op='add', output_use_bias=True, apply_class_weight=False, optimizer='auto', loss='auto', dnn_params={'hidden_units': ((256, 0.3, True), (256, 0.3, True)), 'dnn_activation': 'relu'}, autoint_params={'num_attention': 3, 'num_heads': 1, 'dropout_rate': 0, 'use_residual': True}, fgcnn_params={'fg_f

04-11 16:47:22 W deeptables.m.preprocessor.py 154 - Column index of X has been converted: Index(['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9',
       'x_10', 'x_11'],
      dtype='object')


04-11 16:47:22 I deeptables.m.preprocessor.py 261 - Preparing features...
04-11 16:47:22 I deeptables.m.preprocessor.py 336 - Preparing features taken 0.013810157775878906s
04-11 16:47:22 I deeptables.m.preprocessor.py 341 - Data imputation...
04-11 16:47:22 I deeptables.m.preprocessor.py 383 - Imputation taken 0.013665437698364258s
04-11 16:47:22 I deeptables.m.preprocessor.py 388 - Categorical encoding...
04-11 16:47:22 I deeptables.m.preprocessor.py 393 - Categorical encoding taken 0.20556426048278809s
04-11 16:47:23 I deeptables.m.preprocessor.py 196 - fit_transform taken 0.26237940788269043s
04-11 16:47:23 I deeptables.m.deeptable.py 353 - Training...
04-11 16:47:23 I deeptables.m.deeptable.py 752 - Injected a callback [EarlyStopping]. monitor:val_rootmeansquarederror, patience:5, mode:min
04-11 16:47:23 I deeptables.u.dataset_generator.py 250 - create dataset generator with _TFDGForPandas, batch_size=128, shuffle=True, drop_remainder=True
04-11 16:47:23 I deeptables.u.dataset_gen

/home/hrn4ch/.conda/envs/myenv/lib/python3.7/site-packages/keras/initializers/initializers_v2.py:121: UserWarning: The initializer RandomUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "


04-11 16:47:23 I deeptables.m.deepmodel.py 105 - training...
Epoch 1/100
1/1 [==============================] - 5s 5s/step - loss: 2.0256 - root_mean_squared_error: 0.4881 - val_loss: 1.3968 - val_root_mean_squared_error: 0.4345
Epoch 2/100
1/1 [==============================] - 0s 50ms/step - loss: 2.2099 - root_mean_squared_error: 0.4918 - val_loss: 1.3943 - val_root_mean_squared_error: 0.4342
Epoch 3/100
1/1 [==============================] - 0s 42ms/step - loss: 1.4595 - root_mean_squared_error: 0.4270 - val_loss: 1.3914 - val_root_mean_squared_error: 0.4338
Epoch 4/100
1/1 [==============================] - 0s 36ms/step - loss: 1.3271 - root_mean_squared_error: 0.4327 - val_loss: 1.3888 - val_root_mean_squared_error: 0.4334
Epoch 5/100
1/1 [==============================] - 0s 47ms/step - loss: 1.3316 - root_mean_squared_error: 0.4275 - val_loss: 1.3861 - val_root_mean_squared_error: 0.4330
Epoch 6/100
1/1 [==============================] - 0s 22ms/step - loss: 1.3170 - root_mean_

04-11 16:47:30 W deeptables.m.preprocessor.py 154 - Column index of X has been converted: Index(['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9',
       'x_10', 'x_11'],
      dtype='object')


04-11 16:47:30 I deeptables.m.preprocessor.py 249 - transform_X taken 0.04905128479003906s
04-11 16:47:30 I deeptables.m.preprocessor.py 230 - Transform [y]...
04-11 16:47:30 I deeptables.m.preprocessor.py 236 - transform_y taken 0.0004649162292480469s
04-11 16:47:30 I deeptables.m.deepmodel.py 158 - Performing evaluation...
04-11 16:47:30 I deeptables.u.dataset_generator.py 250 - create dataset generator with _TFDGForPandas, batch_size=512, shuffle=False, drop_remainder=False
04-11 16:47:30 I deeptables.m.deeptable.py 685 - Perform prediction...
04-11 16:47:30 I deeptables.m.preprocessor.py 242 - Transform [X]...


04-11 16:47:30 W deeptables.m.preprocessor.py 154 - Column index of X has been converted: Index(['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9',
       'x_10', 'x_11'],
      dtype='object')


04-11 16:47:30 I deeptables.m.preprocessor.py 249 - transform_X taken 0.04181814193725586s
04-11 16:47:30 I deeptables.m.deepmodel.py 130 - Performing predictions...
04-11 16:47:30 I deeptables.u.dataset_generator.py 250 - create dataset generator with _TFDGForPandas, batch_size=128, shuffle=False, drop_remainder=False
04-11 16:47:30 I deeptables.m.deeptable.py 559 - predict_proba taken 0.2717902660369873s
04-11 16:47:30 I deeptables.m.deeptable.py 594 - Reverse indicators to labels.


{'loss': 1.4419238567352295, 'root_mean_squared_error': 0.4393136501312256}

In [38]:
dt_bio = deeptable.DeepTable(config=conf)
dt_bio.fit(X_train_bio_tensor, y_train_bio['Biochar'], epochs=100)
score = dt_bio.evaluate(X_test_bio_tensor, y_test_bio['Biochar'], batch_size=512, verbose=0)
preds = dt_bio.predict(X_test_bio_tensor)
score

04-11 16:47:36 I deeptables.m.deeptable.py 337 - X.Shape=torch.Size([67, 12]), y.Shape=(67,), batch_size=128, config=ModelConfig(name='conf-1', nets=['dcn_nets'], categorical_columns='auto', exclude_columns=[], task='auto', pos_label=None, metrics=['RootMeanSquaredError'], auto_categorize=False, cat_exponent=0.5, cat_remain_numeric=True, auto_encode_label=True, auto_imputation=True, auto_discrete=False, auto_discard_unique=True, apply_gbm_features=False, gbm_params={}, gbm_feature_type='embedding', fixed_embedding_dim=True, embeddings_output_dim=4, embeddings_initializer='uniform', embeddings_regularizer=None, embeddings_activity_regularizer=None, dense_dropout=0, embedding_dropout=0.3, stacking_op='add', output_use_bias=True, apply_class_weight=False, optimizer='auto', loss='auto', dnn_params={'hidden_units': ((256, 0.3, True), (256, 0.3, True)), 'dnn_activation': 'relu'}, autoint_params={'num_attention': 3, 'num_heads': 1, 'dropout_rate': 0, 'use_residual': True}, fgcnn_params={'fg_f

04-11 16:47:36 W deeptables.m.preprocessor.py 154 - Column index of X has been converted: Index(['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9',
       'x_10', 'x_11'],
      dtype='object')


04-11 16:47:36 I deeptables.m.preprocessor.py 261 - Preparing features...
04-11 16:47:36 I deeptables.m.preprocessor.py 336 - Preparing features taken 0.013606071472167969s
04-11 16:47:36 I deeptables.m.preprocessor.py 341 - Data imputation...
04-11 16:47:36 I deeptables.m.preprocessor.py 383 - Imputation taken 0.01352238655090332s
04-11 16:47:36 I deeptables.m.preprocessor.py 388 - Categorical encoding...
04-11 16:47:36 I deeptables.m.preprocessor.py 393 - Categorical encoding taken 0.12623929977416992s
04-11 16:47:36 I deeptables.m.preprocessor.py 196 - fit_transform taken 0.1714184284210205s
04-11 16:47:36 I deeptables.m.deeptable.py 353 - Training...
04-11 16:47:36 I deeptables.m.deeptable.py 752 - Injected a callback [EarlyStopping]. monitor:val_rootmeansquarederror, patience:5, mode:min
04-11 16:47:36 I deeptables.u.dataset_generator.py 250 - create dataset generator with _TFDGForPandas, batch_size=128, shuffle=True, drop_remainder=True
04-11 16:47:36 I deeptables.u.dataset_gener

/home/hrn4ch/.conda/envs/myenv/lib/python3.7/site-packages/keras/initializers/initializers_v2.py:121: UserWarning: The initializer RandomUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "


04-11 16:47:36 I deeptables.m.deepmodel.py 287 - >>>>>>>>>>>>>>>>>>>>>> Model Desc <<<<<<<<<<<<<<<<<<<<<<< 
---------------------------------------------------------
inputs:
---------------------------------------------------------
['all_categorical_vars: (12)']
---------------------------------------------------------
embeddings:
---------------------------------------------------------
input_dims: [69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69]
output_dims: [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
dropout: 0.3
---------------------------------------------------------
dense: dropout: 0
batch_normalization: False
---------------------------------------------------------
concat_embed_dense: shape: (None, 48)
---------------------------------------------------------
nets: ['dcn_nets']
---------------------------------------------------------
dcn-widecross: input_shape (None, 48), output_shape (None, 48)
dcn-dnn2: input_shape (None, 48), output_shape (None, 256)
dcn: input_shape (None, 48), 

04-11 16:47:43 W deeptables.m.preprocessor.py 154 - Column index of X has been converted: Index(['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9',
       'x_10', 'x_11'],
      dtype='object')


04-11 16:47:43 I deeptables.m.preprocessor.py 249 - transform_X taken 0.04109668731689453s
04-11 16:47:43 I deeptables.m.preprocessor.py 230 - Transform [y]...
04-11 16:47:43 I deeptables.m.preprocessor.py 236 - transform_y taken 0.0002887248992919922s
04-11 16:47:43 I deeptables.m.deepmodel.py 158 - Performing evaluation...
04-11 16:47:43 I deeptables.u.dataset_generator.py 250 - create dataset generator with _TFDGForPandas, batch_size=512, shuffle=False, drop_remainder=False
04-11 16:47:43 I deeptables.m.deeptable.py 685 - Perform prediction...
04-11 16:47:43 I deeptables.m.preprocessor.py 242 - Transform [X]...


04-11 16:47:43 W deeptables.m.preprocessor.py 154 - Column index of X has been converted: Index(['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9',
       'x_10', 'x_11'],
      dtype='object')


04-11 16:47:43 I deeptables.m.preprocessor.py 249 - transform_X taken 0.0414271354675293s
04-11 16:47:43 I deeptables.m.deepmodel.py 130 - Performing predictions...
04-11 16:47:43 I deeptables.u.dataset_generator.py 250 - create dataset generator with _TFDGForPandas, batch_size=128, shuffle=False, drop_remainder=False
04-11 16:47:43 I deeptables.m.deeptable.py 559 - predict_proba taken 0.41271424293518066s
04-11 16:47:43 I deeptables.m.deeptable.py 594 - Reverse indicators to labels.


{'loss': 1.469782829284668, 'root_mean_squared_error': 0.44650277495384216}